# Setting up 

In [123]:
# Necessary import statements:
!pip install lenskit
!git clone https://github.com/crash-course-ai/lab4-recommender-systems.git

link='http://files.grouplens.org/datasets/movielens/ml-100k.zip'

!wget -nc http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip

import lenskit.datasets as ds
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras import backend
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer

fatal: destination path 'lab4-recommender-systems' already exists and is not an empty directory.
File ‘ml-100k.zip’ already there; not retrieving.

Archive:  ml-100k.zip


In [124]:
# save movie data
movies = ds.MovieLens('lab4-recommender-systems/')

In [125]:
# use a different data set as well for model 2:
# overall_stats = pd.read_csv('ml-100k/u.info', header=None)

# set up df for ratings data set:
col_one = ['user id','movie id','rating','timestamp']
ratings_dataset = pd.read_csv('ml-100k/u.data', sep='\t',header=None,names=col_one) # it is a tab seperated file

# set up df for items which is not tab seperated, but seperated by '|':
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_two = d.split(' | ')
items_dataset = pd.read_csv('ml-100k/u.item', sep='|',header=None,names=column_two,encoding='latin-1')
movie_dataset = items_dataset[['movie id','movie title']]

# Data Exploration

In [126]:
# look at the ratings data:
print(len(movies.ratings))
movies.ratings.head(100)
hi = movies.ratings.item

100836


There are 100,836 total entries and 610 different users, with an average of 165 ratings per user. \\
The first 222 entries are made up of user 1 ratings, and user 2 starts and 223 and so on.. \\
This shows that there is a lot of data per user to be played with.

In [127]:
# look at the movies data:
print(len(movies.movies))
movies.movies.head(100)

9742


,title,genres
item,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
107,Muppet Treasure Island (1996),Adventure|Children|Comedy|Musical
108,Catwalk (1996),Documentary
110,Braveheart (1995),Action|Drama|War


The data seems to not have missing values. \\
There are a total of 9,742 movies in this data set. \\
Every movie is correlated to an item, which must be the item that is being rated in the 'ratings' data

In [128]:
# look at the tags data:
print(len(movies.tags))
movies.tags.head(100)

3683


,user,item,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
95,62,6541,captain nemo,1525554054
96,62,6541,comic book,1525554028
97,62,6541,gothic,1525554056
98,62,6541,Peta Wilson,1525554040


The data seems to not have missing values. \\
There are a total of 3,683 tags in this data set. \\
A tag is what this movie is associted with. It is interesting to not that a tag can range from a character in the movie, to theme, to acto in movie, etc. May cause problems down line to have such a generic column of data. 

Item is correlatd to the item in movie data which is correlated to movie.

In [129]:
# look at the links data:
print(len(movies.links))
movies.links.head(100)

9742


,imdbId,tmdbId
item,,
1,114709,862
2,113497,8844
3,113228,15602
4,114885,31357
5,113041,11862
...,...,...
107,117110,10874
108,112646,89333
110,112573,197


There are 9,742 rows of data which makes sense since this links the movie to the imdb reference so there must be the same amount of rows

This data seems like it will not require cleaning, and am not sure how important it will be to the recommender system, seems like it is here just to reference.

# Base line model
Create a baseline model through using mean of ratings as specified in requirements doc

In [130]:
# take the average rating of each movie:
average_ratings = movies.ratings.groupby(['item'], as_index=False).mean()

# sort the ratings to get the highest rated movies on top:
sorted_average_ratings = average_ratings.sort_values(by="rating", ascending=False)

# do a join on item to movie title so that we know what each rating corresponds to:
baseline_recommender_df = sorted_average_ratings.merge(movies.movies['title'], on='item')

baseline_recommender = baseline_recommender_df['title']

# display base line recommender: Since it is base line, it would work for any user.
print(baseline_recommender.head())

# the movies may be unknown because there is no specification on how many people myst rate a movie. So one perfect rating could potentially be at the top of the list. Since this is just a baseline model it does not have to be imporved, per the requirements.

0    Paper Birds (Pájaros de papel) (2010)
1               Act of Killing, The (2012)
2                          Jump In! (2007)
3                             Human (2015)
4                      L.A. Slasher (2015)
Name: title, dtype: object


# Simple Model -- Illustrate predicting movies per user
Collabarative Filtering using a model based approach. I will use a model based approach as it does not demand as much memory storage, and will take advantage of dimensionality reduction.

In [9]:
# find num of users and items to recommend based on user to user and item to item similarity:
num_users = len(pd.unique(movies.ratings['user']) )
num_items = len(pd.unique(movies.ratings['item']) )

print('number of users:', num_users)
print('number of items:', num_items)

number of users: 610
number of items: 9724


In [10]:
# train/test split the movies:
X_train, X_test, y_train, y_test = train_test_split(movies.ratings, movies.ratings, test_size=0.33) # I dont require the y variables since there is no pre-determined correct output. just setting it so i don't get an error.

Find similarity between users and items

In [11]:
# before creating a user-item matrix I must make the items range from 0-5,973  to be able to have it in a way that I can reference it as an index in the user-item matrix:
item_to_index = {}
counter = 0
for item in pd.unique(movies.ratings['item']):
  item_to_index[item] = counter
  counter += 1
# first must create a user-item matrix:
user_item_matrix = np.zeros((num_users, num_items))
for line in movies.ratings.itertuples():
  user_item_matrix[line.user-1, item_to_index.get(line.item)] = line.rating # subtract 1 to adjust to a zero indexed array.
  # user_item_matrix[line.user-1, line.item%line.user] = line.rating # subtract 1 to adjust to a zero indexed array.

In [12]:
# find similarity:
user_similarity = pairwise_distances(user_item_matrix, metric='cosine')

In [13]:
# This code will predict what movies a user will like through using collabarative filtering
def predict(ratings, similarity):
    mean_user_rating = ratings.mean(axis=1)
    ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
    pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    return pred

In [14]:
# predictions of how much a user will like every movie, and outputs a matrix representing every user:
user_prediction = predict(user_item_matrix, user_similarity)

In [15]:
# dictionary to easily get movie title from item(movie id):
item_to_title = {}
for line in movies.movies.itertuples():
  item_to_title[line.Index] = line.title

# recommend movies:
def recommend_k_movies_to_user(user, k):
  # find indices of movies to recommend:
  user_preds = user_prediction[user-1]
  # find 5 lowest indices:
  k_preds = []
  k_preds = (np.argsort(user_preds, axis=0)[0:5])
  print(k_preds)
  # convert those indices to movie ids(items):
  movie_ids = []
  for index in k_preds:
    movie_ids.append(item_to_index.get(index))
  # convert movie ids to movie titles:
  print(movie_ids)
  movie_names = []
  # for id in movie_ids:
  #   movie_names.append(item_to_title.get(id))
  return movie_ids

In [16]:
# get the top k movie suggestions per user:
def get_recs(user_prediction, user, number_of_suggestions):
  # user the user matrix to suggest a movie
  user_index = []
  for i in range(num_users):
    user_index.append(np.argsort(user_prediction[i], axis=0)[:])
  # get movie titles that should be recommended to user:
  recs = []
  for person in user_index:
    user_pred = []
    for i in person:
      if len(user_pred) == number_of_suggestions:
        break
      if item_to_title.get(i) != None:
        user_pred.append(item_to_title.get(i) )
    recs.append(user_pred)
  return recs[user-1] # return user -1 as users are 1-indexed but the list is zero indexed

# evaluate function: show that it outputs movies
print(get_recs(user_prediction, 382, 1))

['Reefer Madness (a.k.a. Tell Your Children) (1938)']


#RMSE Simple Model 1 (Model 2 is my main model)

I ran into a problem trying to munge this data to fit a keras model to calculate the RMSE score so I did a similar model using a different data set in the next dropdown. I am still leaving this code as it is still functional in predicting movies and shows an alternative method. Additionally it uses the above drop down, which I found very interesting.

In [18]:
merged_dataset = pd.merge(movies.ratings, movies.movies, how='inner', on='item')

refined_dataset = merged_dataset.groupby(by=['user','title'], as_index=False).agg({"rating":"mean"})

In [19]:
# clean the data so the join is good
merged_dataset = pd.merge(movies.ratings, movies.movies, how='inner', on='item')

# use a label encoder 
label_encoder = LabelEncoder()
refined_dataset['user'] = label_encoder.fit_transform(refined_dataset['user'].values)

# get the number of users:
n_users = refined_dataset['user'].nunique()

# encode the items (helps unify):
item_enc = LabelEncoder()
refined_dataset['movie'] = item_enc.fit_transform(refined_dataset['title'].values)

# find number of movies
n_movies = refined_dataset['movie'].nunique()

refined_dataset['rating'] = refined_dataset['rating'].values.astype(np.float32)
min_rating = min(refined_dataset['rating'])
max_rating = max(refined_dataset['rating'])
n_users, n_movies, min_rating, max_rating

(610, 9719, 0.5, 5.0)

In [31]:
# train/test split:
X = refined_dataset[['user', 'movie']].values
y = refined_dataset['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# regularize X indices
new_x = []
prev_elem = None
counter = 0
for elem in X:
  temp_elem = []
  temp_elem.append(elem[0])
  if elem[1] != prev_elem:
    counter += 1
  temp_elem.append(counter)
  new_x.append(temp_elem)
  prev_elem = elem[1]
new_x = np.array(new_x)

y = refined_dataset['rating'].values
X_train, X_test, y_train, y_test = train_test_split(new_x, y, test_size=0.1, random_state=50)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((90748, 2), (10084, 2), (90748,), (10084,))

In [32]:
# munge the train/test split:
n_factors = 150
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]
X_train, X_train_array, X_train_array[0].shape
y_train = (y_train - min_rating)/(max_rating - min_rating)
y_test = (y_test - min_rating)/(max_rating - min_rating)

In [33]:
# function to use as a custom metric in my model since rmse score is required:
def rmse(y_true, y_pred):
  return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [34]:
# user = tf.keras.layers.Input(shape = (1,))
# u = keras.layers.embeddings.Embedding(n_users, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer = tf.keras.regularizers.l2(1e-6))(user)
# u = tf.keras.layers.Reshape((n_factors,))(u)
# movie = tf.keras.layers.Input(shape = (1,))
# m = keras.layers.embeddings.Embedding(n_movies, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(movie)
# m = tf.keras.layers.Reshape((n_factors,))(m)
# x = tf.keras.layers.Activation(activation='relu')(x)
# x = tf.keras.layers.Dense(9)(x)
# x = tf.keras.layers.Activation(activation='softmax')(x)
# model = tf.keras.models.Model(inputs=[user,movie], outputs=x)
# model.compile(optimizer='sgd', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[rmse])

In [38]:
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=3, min_lr=0.000001, verbose=1)

# history = model.fit(x = X_train_array, y = y_train, batch_size=128, epochs=5, verbose=1, validation_data=(X_test_array, y_test),shuffle=True,callbacks=[reduce_lr])

# Simple Model 2 & RMSE

In [131]:
# clean the new data set:
joined_dataset = pd.merge(ratings_dataset, movie_dataset, how='inner', on='movie id')
joined_dataset = joined_dataset.groupby(by=['user id','movie title'], as_index=False).agg({"rating":"mean"})

In [132]:
# clean the data so that all necessary data is in one dataframe with accurate movie ids, instead of unordered numbers which is error prone.
joined_dataset = pd.merge(ratings_dataset, movie_dataset, how='inner', on='movie id')

# use a label encoder 
label_encoder = LabelEncoder()
joined_dataset['user'] = label_encoder.fit_transform(joined_dataset['user id'].values)

# get the number of users:
n_users = joined_dataset['user'].nunique()

# encode the items (helps unify):
item_enc = LabelEncoder()
joined_dataset['movie'] = item_enc.fit_transform(joined_dataset['movie title'].values)

# find number of unique movies
n_movies = joined_dataset['movie'].nunique()

# make calclulations necessary for the model to run, and to output accurate order of movie recommendations:
joined_dataset['rating'] = joined_dataset['rating'].values.astype(np.float32)
min_rating = min(joined_dataset['rating'])
max_rating = max(joined_dataset['rating'])
n_users, n_movies, min_rating, max_rating

(943, 1664, 1.0, 5.0)

In [133]:
# split the data set 80/20:
X = joined_dataset[['user', 'movie']].values
y = joined_dataset['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# clean up split data:
n_factors = 150
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]
X_train, X_train_array, X_train_array[0].shape
y_train = (y_train - min_rating)/(max_rating - min_rating)
y_test = (y_test - min_rating)/(max_rating - min_rating)

In [134]:
# create a rmse function to test the model using a rmse metric:
def rmse(y_true, y_pred):
  return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [135]:
# create the model with the following features:
user = tf.keras.layers.Input(shape = (1,))
u = keras.layers.embeddings.Embedding(n_users, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer = tf.keras.regularizers.l2(1e-6))(user)
u = tf.keras.layers.Reshape((n_factors,))(u)
movie = tf.keras.layers.Input(shape = (1,))
m = keras.layers.embeddings.Embedding(n_movies, n_factors, embeddings_initializer = 'he_normal', embeddings_regularizer=tf.keras.regularizers.l2(1e-6))(movie)
m = tf.keras.layers.Reshape((n_factors,))(m)
x = tf.keras.layers.Concatenate()([u,m])
x = tf.keras.layers.Activation(activation='relu')(x)
x = tf.keras.layers.Dropout(0.05)(x)
x = tf.keras.layers.Dense(9)(x)
x = tf.keras.layers.Activation(activation='softmax')(x)

# instantiate the model, and use a custom rmse function implemented above to test the recommender system in accords with the requirement documentation:
model = tf.keras.models.Model(inputs=[user,movie], outputs=x)
model.compile(optimizer='sgd', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[rmse])

In [136]:
# running low epochs to not spam professor with output. RMSE is currently between 0.5 and 0.6 which is quite low, in consideration for extra credit.
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=3, min_lr=0.000001, verbose=1)
history = model.fit(x = X_train_array, y = y_train, batch_size=128, epochs=5, verbose=1, validation_data=(X_test_array, y_test),shuffle=True,callbacks=[reduce_lr])

Epoch 1/5
625/625 [==============================] - 4s 5ms/step - loss: 1.5785 - rmse: 0.5453 - val_loss: 0.8154 - val_rmse: 0.5788
Epoch 2/5
625/625 [==============================] - 4s 6ms/step - loss: 0.7635 - rmse: 0.5834 - val_loss: 0.6718 - val_rmse: 0.5926
Epoch 3/5
625/625 [==============================] - 4s 6ms/step - loss: 0.6552 - rmse: 0.5936 - val_loss: 0.6161 - val_rmse: 0.5960
Epoch 4/5
625/625 [==============================] - 3s 6ms/step - loss: 0.6083 - rmse: 0.5965 - val_loss: 0.5878 - val_rmse: 0.5977
Epoch 5/5
625/625 [==============================] - 3s 5ms/step - loss: 0.5845 - rmse: 0.5972 - val_loss: 0.5711 - val_rmse: 0.5988


In [137]:
# get recommended movies for users:
def get_k_rec_movies_for_a_user(user, k):
  # get recommended movies for any user:
  encoded_user_id = label_encoder.transform([user])

  # to be able to predict which movies a given user would like, then we must know which movies has and has not seen:
  seen_movies = list(joined_dataset[joined_dataset['user id'] == user]['movie'])
  unseen_movies = [i for i in range(min(joined_dataset['movie']), max(joined_dataset['movie'])+1) if i not in seen_movies]

  # this is what will be inputted into the model:
  model_input = [np.asarray(list(encoded_user_id)*len(unseen_movies)), np.asarray(unseen_movies)]

  # pass data through the DNN
  predicted_ratings = model.predict(model_input)
  predicted_ratings = np.max(predicted_ratings, axis=1)

  # sort predictions to only get top movie suggestions:
  sorted_index = np.argsort(predicted_ratings)[::-1]

  # return k movie suggestions:
  return item_enc.inverse_transform(sorted_index)[:k]

# evaluate the function:
print('user 1:', get_k_rec_movies_for_a_user(1, 5) )# get five movie suggestions for user 1.
print('user 100:', get_k_rec_movies_for_a_user(100, 10) )# get ten movie suggestions for user 100.

user 1: ['Pather Panchali (1955)' 'Pillow Book, The (1995)'
 'Paris Is Burning (1990)' 'Nixon (1995)' 'Crossing Guard, The (1995)']
user 100: ['Selena (1997)' 'Circle of Friends (1995)' 'Shooting Fish (1997)'
 'Search for One-eye Jimmy, The (1996)'
 'Rendezvous in Paris (Rendez-vous de Paris, Les) (1995)'
 'Diabolique (1996)' 'North by Northwest (1959)' 'Michael Collins (1996)'
 '187 (1997)' 'Houseguest (1994)']


# Hybrid Model

In [138]:
# in preparation for the hybrid model I will fist build a content based model using TF/IDF:

# join useful info:
content_df = pd.merge(movies.movies, movies.ratings, how='inner', on='item')

# drop useless info:
content_df.drop(columns=['timestamp'], inplace=True)  # Remove useless features

# group data by user:
group_by_user = content_df.groupby(['user'])

# get all reviews per person:
reviews_per_person = []
for i, j in group_by_user:
    reviews_per_person.append(j['rating'].values)  # Extract target feature (ratings) from sorted data

In [139]:
# Creating a list of dataframes with the feature set (movie info about genres and release year), grouped by userID
content_df.drop(columns=['rating'], inplace=True)
x_grouped_by_user = content_df.groupby(['user'])

x_train_listed = []
for user_id, group in x_grouped_by_user:
    x_train_listed.append(group.drop(columns='user'))
    
# The feature set for the first user
x_train_listed[0]

,item,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
325,3,Grumpier Old Men (1995),Comedy|Romance
433,6,Heat (1995),Action|Crime|Thriller
2107,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
2379,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...
56816,3744,Shaft (2000),Action|Crime|Thriller
57276,3793,X-Men (2000),Action|Adventure|Sci-Fi
57457,3809,What About Bob? (1991),Comedy
59170,4006,Transformers: The Movie (1986),Adventure|Animation|Children|Sci-Fi


In [167]:
# use tf/idf on movie genres:
tfidf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4) for c in combinations(s.split('|'), r=i)))
tfidf_matrix = tfidf.fit_transform(movies.movies['genres'])

# find cosine similarity to be able to compare scores and flatten from matrix:
cosine_sim = cosine_similarity(tfidf_matrix)

# transform cosine similarity to a pd dataframe for similarity:
cosine_sim_df = pd.DataFrame(cosine_sim, columns = movies.movies['title'].values)

def genre_recommendations(i, M, items, k):
  ix = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
  closest = M.columns[ix[-1:-(k+2):-1]]
  closest = closest.drop(i, errors='ignore')
  return pd.DataFrame(closest)

# Evaluate TF/IDF:
print(genre_recommendations('Toy Story (1995)', cosine_sim_df, movies.movies[['title', 'genres']], 100) )

                                                    0
0                                        Moana (2016)
1   Asterix and the Vikings (Astérix et les Viking...
2                              Shrek the Third (2007)
3                            The Good Dinosaur (2015)
4                                  Toy Story 2 (1999)
..                                                ...
95                                      Robots (2005)
96  Spirited Away (Sen to Chihiro no kamikakushi) ...
97                                        Epic (2013)
98                 South Park: Imaginationland (2008)
99                                   Anomalisa (2015)

[100 rows x 1 columns]


In [168]:
# get movies from the simple model and stack the content based model on top:
def hybrid(user, sim_movie, k_movies):

  # get all movie suggestions
  collab_model = get_k_rec_movies_for_a_user(1, 9000) 
  # collab_model = collab_model.split('\n')

  # filter movies to get similar movie the person wants: content based requires genre specification for my model:
  genre_recs = genre_recommendations(sim_movie, cosine_sim_df, movies.movies[['title', 'genres']], 3*k_movies)[0]

  # add content based recs to a dictionary:
  content_based_recs = {}
  for movie in genre_recs:
    content_based_recs[movie] = 1

  # combine both together:
  combiner = []
  for i in range(len(collab_model)):
    if len(combiner) < k_movies:
      if collab_model[i] == genre_recs[i]:
        combiner.append(collab_model[i])
      elif genre_recs[i] != collab_model[i]:
        combiner.append(collab_model[i])
        combiner.append(genre_recs[i])
  return combiner

# evaluta hybrid model:
hybrid_result = (hybrid(1, 'Shrek the Third (2007)', 4))

history = model.fit(x = X_train_array, y = y_train, batch_size=128, epochs=8, verbose=1, validation_data=(X_test_array, y_test),shuffle=True,callbacks=[reduce_lr])

print(hybrid_result)

Epoch 1/8
625/625 [==============================] - 3s 5ms/step - loss: 0.4725 - rmse: 0.6059 - val_loss: 0.4754 - val_rmse: 0.6057
Epoch 2/8
625/625 [==============================] - 3s 5ms/step - loss: 0.4710 - rmse: 0.6058 - val_loss: 0.4740 - val_rmse: 0.6059
Epoch 3/8
625/625 [==============================] - 3s 5ms/step - loss: 0.4695 - rmse: 0.6058 - val_loss: 0.4726 - val_rmse: 0.6062
Epoch 4/8
625/625 [==============================] - 3s 5ms/step - loss: 0.4680 - rmse: 0.6061 - val_loss: 0.4713 - val_rmse: 0.6062
Epoch 5/8
625/625 [==============================] - 3s 5ms/step - loss: 0.4665 - rmse: 0.6063 - val_loss: 0.4700 - val_rmse: 0.6061
Epoch 6/8
625/625 [==============================] - 3s 5ms/step - loss: 0.4651 - rmse: 0.6062 - val_loss: 0.4687 - val_rmse: 0.6064
Epoch 7/8
625/625 [==============================] - 3s 5ms/step - loss: 0.4638 - rmse: 0.6065 - val_loss: 0.4675 - val_rmse: 0.6064
Epoch 8/8
625/625 [==============================] - 3s 5ms/step - lo